In [19]:
from transformers import AutoTokenizer, TFAutoModel, AutoModel, TFBertForSequenceClassification, TFAutoModelForSequenceClassification, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer
import tensorflow as tf
from models import *
import numpy as np

import transformers
import shap
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
import scipy
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torchsummary import summary
import torch.nn.functional as F


# HateSpeachModel

In [21]:
hate_tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert")

hate_model = AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert")

pred = transformers.pipeline("text-classification", model=hate_model, tokenizer=hate_tokenizer, return_all_scores=True)

In [22]:
type(hate_tokenizer)

transformers.models.bert.tokenization_bert_fast.BertTokenizerFast

In [23]:
type(hate_model)

transformers.models.bert.modeling_bert.BertForSequenceClassification

In [24]:
hate_tokenizer.vocab_size

30522

In [30]:
hate_tokenizer.encode("This is an example sentence")

[101, 2023, 2003, 2019, 2742, 6251, 102]

In [36]:
hate_vocab = hate_tokenizer.decode(range(0, hate_tokenizer.vocab_size))
hate_vocab.

'[PAD] [unused0] [unused1] [unused2] [unused3] [unused4] [unused5] [unused6] [unused7] [unused8] [unused9] [unused10] [unused11] [unused12] [unused13] [unused14] [unused15] [unused16] [unused17] [unused18] [unused19] [unused20] [unused21] [unused22] [unused23] [unused24] [unused25] [unused26] [unused27] [unused28] [unused29] [unused30] [unused31] [unused32] [unused33] [unused34] [unused35] [unused36] [unused37] [unused38] [unused39] [unused40] [unused41] [unused42] [unused43] [unused44] [unused45] [unused46] [unused47] [unused48] [unused49] [unused50] [unused51] [unused52] [unused53] [unused54] [unused55] [unused56] [unused57] [unused58] [unused59] [unused60] [unused61] [unused62] [unused63] [unused64] [unused65] [unused66] [unused67] [unused68] [unused69] [unused70] [unused71] [unused72] [unused73] [unused74] [unused75] [unused76] [unused77] [unused78] [unused79] [unused80] [unused81] [unused82] [unused83] [unused84] [unused85] [unused86] [unused87] [unused88] [unused89] [unused90] [u

# SentenceSimilarity

In [32]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
ss_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-xlm-r-multilingual-v1')
ss_model = AutoModel.from_pretrained('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

# Tokenize sentences
encoded_input = ss_tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = ss_model(**encoded_input)

# Perform pooling. In this case, max pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("Sentence embeddings:")
print(sentence_embeddings)

Sentence embeddings:
tensor([[ 0.1145,  0.0768,  0.0263,  ..., -0.1323, -0.0056,  0.3162],
        [ 0.0065,  0.1654, -0.0364,  ...,  0.1892,  0.2014,  0.2443]])


In [33]:
ss_tokenizer.encode("This is an example sentence")

[0, 3293, 83, 142, 27781, 149357, 2]

In [34]:
ss_tokenizer.vocab_size

250002

In [ ]:
np.sqrt(np.sum((sentence_embeddings[0] - sentence_embeddings[1])**2))

In [37]:
ss_tokenizer.decode(range(0, ss_tokenizer.vocab_size))

'<s><pad></s><unk>,. s de- aa:ei () itn - la en in na\'’... e на。o?enu и o、!m se quer的 " di – to da в، un “y do jeer sa"а og за A”/ وante die да thed erin; una не si ja za v et is уdane I elиes skni « le l z on at for _taе dу 1re ne på no ofde y се du 2 perti yang te para der dan som।eth med пом van(le të с and „ 3 tilیl anla alja delar w detli ya :...の“ kano con S pokaas meis und su» denzj omsi ve och درهg بهي O د har отse av be arto і 5onma ir).em porо es les ki 4ul kan ‘таch ।ga 10 з ilна在ing και کےit еra una soтus des hetтеы ku od من\xados op ngςje asc La في E\'attхяdo), / paةを а àp до haatا waі af De bir和komi 2018] nie ازstvenya了b Mcomَ vi peがme ésA var um के کیal che ma — eenS az јеed kony nikiamн2 6om va taun V keй C cu nuum میںcikular din nëن mi და воにтоcevakan man biv Bне bo од L » Kはekak U în m ang էke non em Na是სको Enни ال In 7 os3 הlo par의ت är tu‘be bu το comم ca [ilно das 8 می T του να 20 los að by D á é föryaاتD youia liիba که zu ce را оca mai sięमा با을 чтокаri В1омf में 

In [42]:
sentences = ['Kocham zabijać', 'Gotowanie to moja pasja']
encoded_input = ss_tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = ss_model(**encoded_input)

# Perform pooling. In this case, max pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print(torch.sqrt(torch.sum((sentence_embeddings[0] - sentence_embeddings[1])**2)))

tensor(5.4686)


In [71]:
hate_to_love = []
for i in range(hate_tokenizer.vocab_size):
    print(ss_tokenizer.decode(ss_tokenizer.encode(hate_tokenizer.decode(i))[1:-1]))
    hate_to_love.append(ss_tokenizer.encode(hate_tokenizer.decode(i))[1:-1])
hate_to_love

[PAD]
[unused0]
[unused1]
[unused2]
[unused3]
[unused4]
[unused5]
[unused6]
[unused7]
[unused8]
[unused9]
[unused10]
[unused11]
[unused12]
[unused13]
[unused14]
[unused15]
[unused16]
[unused17]
[unused18]
[unused19]
[unused20]
[unused21]
[unused22]
[unused23]
[unused24]
[unused25]
[unused26]
[unused27]
[unused28]
[unused29]
[unused30]
[unused31]
[unused32]
[unused33]
[unused34]
[unused35]
[unused36]
[unused37]
[unused38]
[unused39]
[unused40]
[unused41]
[unused42]
[unused43]
[unused44]
[unused45]
[unused46]
[unused47]
[unused48]
[unused49]
[unused50]
[unused51]
[unused52]
[unused53]
[unused54]
[unused55]
[unused56]
[unused57]
[unused58]
[unused59]
[unused60]
[unused61]
[unused62]
[unused63]
[unused64]
[unused65]
[unused66]
[unused67]
[unused68]
[unused69]
[unused70]
[unused71]
[unused72]
[unused73]
[unused74]
[unused75]
[unused76]
[unused77]
[unused78]
[unused79]
[unused80]
[unused81]
[unused82]
[unused83]
[unused84]
[unused85]
[unused86]
[unused87]
[unused88]
[unused89]
[unused90]
[un

[unused891]
[unused892]
[unused893]
[unused894]
[unused895]
[unused896]
[unused897]
[unused898]
[unused899]
[unused900]
[unused901]
[unused902]
[unused903]
[unused904]
[unused905]
[unused906]
[unused907]
[unused908]
[unused909]
[unused910]
[unused911]
[unused912]
[unused913]
[unused914]
[unused915]
[unused916]
[unused917]
[unused918]
[unused919]
[unused920]
[unused921]
[unused922]
[unused923]
[unused924]
[unused925]
[unused926]
[unused927]
[unused928]
[unused929]
[unused930]
[unused931]
[unused932]
[unused933]
[unused934]
[unused935]
[unused936]
[unused937]
[unused938]
[unused939]
[unused940]
[unused941]
[unused942]
[unused943]
[unused944]
[unused945]
[unused946]
[unused947]
[unused948]
[unused949]
[unused950]
[unused951]
[unused952]
[unused953]
[unused954]
[unused955]
[unused956]
[unused957]
[unused958]
[unused959]
[unused960]
[unused961]
[unused962]
[unused963]
[unused964]
[unused965]
[unused966]
[unused967]
[unused968]
[unused969]
[unused970]
[unused971]
[unused972]
[unused973]
[unu

range
hours
peter
arm
##da
morning
dr
sold
##ry
quickly
directed
1993
guitar
china
##w
31
list
##ma
performed
media
uk
players
smile
##rs
myself
40
placed
coach
province
towards
wouldn
leading
whole
boy
official
designed
grand
census
##el
europe
attack
japanese
henry
1991
##re
##os
cross
getting
alone
action
lower
network
wide
washington
japan
1990
hospital
believe
changed
sister
##ar
hold
gone
sir
hadn
ship
##ka
studies
academy
shot
rights
below
base
bad
involved
kept
largest
##ist
bank
future
especially
beginning
mark
movement
section
female
magazine
plan
professor
lord
longer
##ian
sat
walked
hill
actually
civil
energy
model
families
size
thus
aircraft
completed
includes
data
captain
##or
fight
vocals
featured
richard
bridge
fourth
1989
officer
stone
hear
##ism
means
medical
groups
management
self
lips
competition
entire
lived
technology
leaving
federal
tournament
bit
passed
hot
independent
awards
kingdom
mary
spent
fine
doesn
reported
##ling
jack
fall
raised
itself
stay
true
studio

rich
pull
##ot
commonly
alive
##za
letters
concept
conducted
wearing
happen
bought
becomes
holy
gets
ocean
defeat
languages
purchased
coffee
occurred
titled
##q
declared
applied
sciences
concert
sounds
jazz
brain
##me
painting
fleet
tax
nick
##ius
michigan
count
animals
leaders
episodes
##line
content
##den
birth
##it
clubs
64
palace
critical
refused
fair
leg
laughed
returning
surrounding
participated
formation
lifted
pointed
connected
rome
medicine
laid
taylor
santa
powers
adam
tall
shared
focused
knowing
yards
entrance
falls
##wa
calling
##ad
sources
chosen
beneath
resources
yard
##ite
nominated
silence
zone
defined
##que
gained
thirty
38
bodies
moon
##ard
adopted
christmas
widely
register
apart
iran
premier
serves
du
unknown
parties
##les
generation
##ff
continues
quick
fields
brigade
quiet
teaching
clothes
impact
weapons
partner
flat
theater
supreme
1938
37
relations
##tor
plants
suffered
1936
wilson
kids
begins
##age
1918
seats
armed
internet
models
worth
laws
400
communities
clas

bond
fifteen
pattern
disappeared
##ney
translated
##zed
lip
arab
capture
interests
insurance
##chi
shifted
cave
prix
warning
sections
courts
coat
plot
smell
feed
golf
favorite
maintain
knife
vs
voted
degrees
finance
quebec
opinion
translation
manner
ruled
operate
productions
choose
musician
discovery
confused
tired
separated
stream
techniques
committed
attend
ranking
kings
throw
passengers
measure
horror
fan
mining
sand
danger
salt
calm
decade
dam
require
runner
##ik
rush
associate
greece
##ker
rivers
consecutive
matthew
##ski
sighed
sq
documents
steam
edited
closing
tie
accused
1905
##ini
islamic
distributed
directors
organisation
bruce
7th
breathing
mad
lit
arrival
concrete
taste
08
composition
shaking
faster
amateur
adjacent
stating
1906
twin
flew
##ran
tokyo
publications
##tone
obviously
ridge
storage
1907
carl
pages
concluded
desert
driven
universities
ages
terminal
sequence
borough
250
constituency
creative
cousin
economics
dreams
margaret
notably
reduce
montreal
mode
17th
ears
s

...

etudes
offender
roast
blackmail
curvature
##woods
vile
309
illicit
suriname
##linson
overture
1685
bubbling
gymnast
tucking
##mming
##ouin
maldives
##bala
gurney
##dda
##eased
##oides
backside
pinto
jars
racehorse
tending
##rdial
baronetcy
wiener
duly
##rke
barbarian
cupping
flawed
##thesis
bertha
pleistocene
puddle
swearing
##nob
##tically
fleeting
prostate
amulet
educating
##mined
##iti
##tler
75th
jens
respondents
analytics
cavaliers
papacy
raju
##iente
##ulum
##tip
funnel
271
disneyland
##lley
sociologist
##iam
2500
faulkner
louvre
menon
##dson
276
##ower
afterlife
mannheim
peptide
referees
comedians
meaningless
##anger
##laise
fabrics
hurley
renal
sleeps
##bour
##icle
breakout
kristin
roadside
animator
clover
disdain
unsafe
redesign
##urity
firth
barnsley
portage
reset
narrows
268
commandos
expansive
speechless
tubular
##lux
essendon
eyelashes
smashwords
##yad
##bang
##claim
craved
sprinted
chet
somme
astor
wrocław
orton
266
bane
##erving
##uing
mischief
##amps
##sund
scaling
terr

1b
antennas
hilarious
cougars
paddington
stats
##eger
breakaway
ipod
reza
authorship
prohibiting
scoffed
##etz
##ttle
conscription
defected
trondheim
##fires
ivanov
keenan
##adan
##ciful
##fb
##slow
locating
##ials
##tford
cadiz
basalt
blankly
interned
rags
rattling
##tick
carpathian
reassured
sync
bum
guildford
iss
staunch
##onga
astronomers
sera
sofie
emergencies
susquehanna
##heard
duc
mastery
vh1
williamsburg
bayer
buckled
craving
##khan
##rdes
bloomington
##write
alton
barbecue
##bians
justine
##hri
##ndt
delightful
smartphone
newtown
photon
retrieval
peugeot
hissing
##monium
##orough
flavors
lighted
relaunched
tainted
##games
##lysis
anarchy
microscopic
hopping
adept
evade
evie
##beau
inhibit
sinn
adjustable
hurst
intuition
wilton
cisco
44th
lawful
lowlands
stockings
thierry
##dalen
##hila
##nai
fates
prank
tb
maison
lobbied
provocative
1724
4a
utopia
##qual
carbonate
gujarati
purcell
##rford
curtiss
##mei
overgrown
arenas
mediation
swallows
##rnik
respectful
turnbull
##hedron
##

##mous
barges
neue
throbbed
cipher
johnnie
##a1
##mpt
outburst
##swick
spearheaded
administrations
c1
heartbreak
pixels
pleasantly
##enay
lombardy
plush
##nsed
bobbie
##hly
reapers
tremor
xiang
minogue
substantive
hitch
barak
##wyl
kwan
##encia
910
obscene
elegance
indus
surfer
bribery
conserve
##hyllum
##masters
horatio
##fat
apes
rebound
psychotic
##pour
iteration
##mium
##vani
botanic
horribly
antiques
dispose
paxton
##hli
##wg
timeless
1704
disregard
engraver
hounds
##bau
##version
looted
uno
facilitates
groans
masjid
rutland
antibody
disqualification
decatur
footballers
quake
slacks
48th
rein
scribe
stabilize
commits
exemplary
tho
##hort
##chison
pantry
traversed
##hiti
disrepair
identifiable
vibrated
baccalaureate
##nnis
csa
interviewing
##iensis
##raße
greaves
wealthiest
343
classed
jogged
£5
##58
##atal
illuminating
knicks
respecting
##uno
scrubbed
##iji
##dles
kruger
moods
growls
raider
silvia
chefs
kam
vr
cree
percival
##terol
gunter
counterattack
defiant
henan
ze
##rasia
##r

##kata
gough
##ciency
lads
soared
##ה
undergoes
deformation
outlawed
rubbish
##arus
##mussen
##nidae
##rzburg
arcs
##ingdon
##tituted
1695
wheelbase
wheeling
bombardier
campground
zebra
##lices
##oj
##bain
lullaby
##ecure
donetsk
wylie
grenada
##arding
##ης
squinting
eireann
opposes
##andra
maximal
runes
##broken
##cuting
##iface
##ror
##rosis
additive
britney
adultery
triggering
##drome
detrimental
aarhus
containment
jc
swapped
vichy
##ioms
madly
##oric
##rag
brant
##ckey
##trix
1560
1612
broughton
rustling
##stems
##uder
asbestos
mentoring
##nivorous
finley
leaps
##isan
apical
pry
slits
substitutes
##dict
intuitive
fantasia
insistent
unreasonable
##igen
##vna
domed
hannover
margot
ponder
##zziness
impromptu
jian
lc
rampage
stemming
##eft
andrey
gerais
whichever
amnesia
appropriated
anzac
clicks
modifying
ultimatum
cambrian
maids
verve
yellowstone
##mbs
conservatoire
##scribe
adherence
dinners
spectra
imperfect
mysteriously
sidekick
tatar
tuba
##aks
##ifolia
distrust
##athan
##zle
c2


[[378, 116908, 268],
 [378, 309, 33740, 2389, 268],
 [378, 309, 33740, 76172],
 [378, 309, 33740, 90173],
 [378, 309, 33740, 96386],
 [378, 309, 33740, 617, 268],
 [378, 309, 33740, 758, 268],
 [378, 309, 33740, 910, 268],
 [378, 309, 33740, 966, 268],
 [378, 309, 33740, 1019, 268],
 [378, 309, 33740, 1126, 268],
 [378, 309, 33740, 963, 268],
 [378, 309, 33740, 1662, 268],
 [378, 309, 33740, 1530, 268],
 [378, 309, 33740, 2681, 268],
 [378, 309, 33740, 2592, 268],
 [378, 309, 33740, 1837, 268],
 [378, 309, 33740, 2485, 268],
 [378, 309, 33740, 2489, 268],
 [378, 309, 33740, 1819, 268],
 [378, 309, 33740, 2947, 268],
 [378, 309, 33740, 1549, 268],
 [378, 309, 33740, 3117, 268],
 [378, 309, 33740, 4015, 268],
 [378, 309, 33740, 3742, 268],
 [378, 309, 33740, 2357, 268],
 [378, 309, 33740, 2588, 268],
 [378, 309, 33740, 4046, 268],
 [378, 309, 33740, 3768, 268],
 [378, 309, 33740, 3882, 268],
 [378, 309, 33740, 4235, 268],
 [378, 309, 33740, 1197, 268],
 [378, 309, 33740, 5016, 268],
 [37

In [72]:
np.array(hate_to_love)

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


array([list([378, 116908, 268]), list([378, 309, 33740, 2389, 268]),
       list([378, 309, 33740, 76172]), ..., list([173894, 12]),
       list([173894, 32]), list([173894, 6087])], dtype=object)

In [73]:
ss_tokenizer.decode(hate_to_love[2000])

'to'

In [94]:
t = 7600

In [95]:
hate_to_love[t]

[1053, 26255]

In [96]:
ss_tokenizer.decode(hate_to_love[t])

'tigers'

In [97]:
ss_tokenizer.decode([1, 2, 3, 4, 5])

'<pad></s><unk>,.'

In [100]:
hate_tokenizer.encode('<pad></s><unk>,.')

[101,
 1026,
 11687,
 1028,
 1026,
 1013,
 1055,
 1028,
 1026,
 4895,
 2243,
 1028,
 1010,
 1012,
 102]